# Onboarding

## [MarshData](https://github.com/suarez96/AP-bIO/blob/main/Data.py) object

In [ ]:
from Data import MarshData, FantasiaData
from Transforms import WPC
import Transforms
from Signal import Signal
import os
import numpy as np

marsh_dataset = [
    MarshData(f"../MARSH/{i}/") for i in os.listdir("../MARSH/") if len(i) == 4
]

## Working with the [Signal](https://github.com/suarez96/AP-bIO/blob/main/Signal.py#L10) object

### Plot the entire ECG for the first sample
Might take some time

In [ ]:
marsh_dataset[0].ECG().plot()

In [ ]:
marsh_dataset[0].ECG().fft(top_freq=125)

### Access the raw ECG data

In [ ]:
import matplotlib.pyplot as plt
marsh_dataset[0].ECG().data.shape

## Working with [Transforms](https://github.com/suarez96/AP-bIO/blob/main/Transforms.py#L54)

### Initialize the transform in question

In [ ]:
crop = Transforms.Crop(start=60, end=69)

### Apply the transform

In [ ]:
marsh_dataset[0].ECG().transform(
    transforms = [crop]
)

### Access the transformed data

In [ ]:
marsh_dataset[0].ECG().transformed_data

### Plot the transformed data

In [ ]:
marsh_dataset[0].ECG().plot(transformed=True)

In [ ]:
### Now Plotting IP to for the same crop
marsh_dataset[0].IP().transform(
    transforms = [crop]
)
marsh_dataset[0].IP().transformed_data
marsh_dataset[0].IP().plot(transformed=True)


In [ ]:
marsh_dataset[0].ECG().plot(transformed=True)

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()

# axis for ECG
ax1.plot(marsh_dataset[0].ECG().transformed_data, color='blue')
ax1.set_ylabel('ECG', color='blue')

# axis for IP
ax2 = ax1.twinx()

ax2.plot(marsh_dataset[0].IP().transformed_data, color='red')
ax2.set_ylabel('IP', color='red')

ax1.set_xlabel('Samples')
plt.title('ECG and IP')
fig.set_size_inches(10, 8)

plt.show()

In [ ]:
from Signal import Signal
from Transforms import MeanSubtraction, Detrend
import Models
import numpy

# FFT
def apply_fft(signal):
    data = signal.data
    data -= np.mean(data)
    fft_data = np.fft.fft(data)
    freq = np.fft.fftfreq(data.shape[0], d=1/250)
    return fft_data, freq

#crop out unwanted frequencies
def crop_frequencies(fft_data, freq, low_cutoff=None, high_cutoff=None):
    cropped_fft = fft_data.copy()
    if low_cutoff is not None:
        cropped_fft[(freq < low_cutoff)] = 0
    if high_cutoff is not None:
        cropped_fft[(freq > high_cutoff)] = 0
    return cropped_fft

# Plotting
def plot_signal(signal, title='Signal', xlabel='Samples', ylabel='Amplitude', xlim=None):
    plt.figure(figsize=(12, 6))
    plt.plot(signal)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xlim)
    plt.grid(True)
    plt.show()

# Main code
ecg = marsh_dataset[0].ECG().transformed_data
plot_signal(ecg, title='Original ECG Signal',xlim=(0, 400))
fft_data, freq = apply_fft(ecg)
plot_signal(signal=abs(fft_data), title='ECG Signal', xlabel='Frequencies')

# cut-off frequencies
low_cutoff = 0.5
high_cutoff = 50.0

# crop
cropped_fft = crop_frequencies(fft_data, freq, low_cutoff=low_cutoff, high_cutoff=high_cutoff)
plot_signal(signal=fft_data, title='ECG Signal', xlabel='Frequencies',xlim=(0, 500))

# inverse FFT
cropped_signal = np.fft.ifft(cropped_fft).real


# original and cropped signals
plot_signal(cropped_signal, title='cropped ECG Signal',xlim=(0, 400))